In [6]:
import bagpy
from tqdm import tqdm
from typing import *
from shapely.geometry import *
from os.path import join, exists
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import geopandas as gpd

In [7]:
worlds_df = pd.read_pickle("/home/agusmazzeo/Documents/UTN/Proyecto/worlds_df.pkl")

In [8]:
worlds_df

,world,world_dir,starting_point,free_area,finished_false,finished_true
evaluation_7,<FrontierExploration.preprocessing.layout.synt...,/home/ramiro/.gazeboworlds/evaluation_7.world,"(44.3730343170481, 45.43376604621937)",1586.841949,True,True
evaluation_1,<FrontierExploration.preprocessing.layout.synt...,/home/ramiro/.gazeboworlds/evaluation_1.world,"(38.13039752687315, 44.08558532764939)",1207.029552,True,True
evaluation_5,<FrontierExploration.preprocessing.layout.synt...,/home/ramiro/.gazeboworlds/evaluation_5.world,"(30.162369766644044, 17.229534857934745)",1470.062424,True,True
evaluation_0,<FrontierExploration.preprocessing.layout.synt...,/home/ramiro/.gazeboworlds/evaluation_0.world,"(47.59203620653866, 21.969882899768308)",1093.128472,True,True
evaluation_9,<FrontierExploration.preprocessing.layout.synt...,/home/ramiro/.gazeboworlds/evaluation_9.world,"(27.146833573506807, 26.35109100999387)",1227.752651,True,True
...,...,...,...,...,...,...
evaluation_194,<FrontierExploration.preprocessing.layout.synt...,/home/ramiro/.gazeboworlds/evaluation_194.world,"(22.244891259682838, 5.356341056526373)",891.868292,False,False
evaluation_195,<FrontierExploration.preprocessing.layout.synt...,/home/ramiro/.gazeboworlds/evaluation_195.world,"(41.545717721344076, 10.354559752761045)",712.452498,False,False
evaluation_196,<FrontierExploration.preprocessing.layout.synt...,/home/ramiro/.gazeboworlds/evaluation_196.world,"(11.235190104396708, 7.134928722218135)",1313.795085,False,False
evaluation_197,<FrontierExploration.preprocessing.layout.synt...,/home/ramiro/.gazeboworlds/evaluation_197.world,"(49.83808475194827, 15.364919697504677)",1454.669774,False,False


In [160]:
class BagpyDataHandler:
    
    def __init__(self, worlds_df: pd.DataFrame, outputs_dir: str):
        self.res_values = {}
        self.report = {}
        self.worlds = {}
        count=0
        for index in tqdm(worlds_df[worlds_df["finished_true"]].index):
            if count==10:
                break
            count+=1
            result = self.get_results_from_folder(join(outputs_dir, index))
            if result is not None:
                self.res_values[index] = result
                self.worlds[index] = worlds_df.loc[index]
                self.report[index] = self.get_report_data(index, result)
                
    def get_report_data(self, index: str, result: Dict[str, Any]):
        report = {}
        world_polygon = self.worlds[index].world.free_space_polygon
        if isinstance(world_polygon, MultiPolygon):
            world_polygon = world_polygon.convex_hull
        for key in result.keys():
            if key not in report.keys():
                report[key] = {}
            poly = gpd.GeoDataFrame(geometry=[world_polygon.exterior])
            report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
        return report
    
    def get_exploration_data(self, key: str, index: str, result: Dict[str, Any]):
        free_area = self.worlds[index]["free_area"]
        explored_per_distance = pd.merge_asof(result[key]["explored_df"], result[key]["travelled_distance_df"], on="timestamp")
        explored_area = explored_per_distance["m_2_explored"].iloc[-1]
        exploration = {
            "explored_percentage": 100*explored_area/free_area,
            "total_time_sec": explored_per_distance.index[-1],
            "dist_explored_20%": explored_per_distance.loc[explored_per_distance["m_2_explored"]<(20*free_area/100)].iloc[-1]["cum_distance"],
            "dist_explored_50%": explored_per_distance.loc[explored_per_distance["m_2_explored"]<(50*free_area/100)].iloc[-1]["cum_distance"],
            "dist_explored_80%": explored_per_distance.loc[explored_per_distance["m_2_explored"]<(80*free_area/100)].iloc[-1]["cum_distance"],
            "dist_total": explored_per_distance.iloc[-1]["cum_distance"],
        }
        return exploration
            
    def get_results_from_folder(self, folder: str) -> Dict[str, Tuple[List[float], List[float]]]:
        try:
            utn_path = join(folder, "UTN")
            maryland_path = join(folder, "MARYLAND")
            if not exists(utn_path) or not exists(maryland_path):
                # print(f"Skipping {folder} because of not having both maryland and UTN")
                return None
            utn_bag_file = glob(join(utn_path, "*.bag"))[0]
            maryland_bag_file = glob(join(maryland_path, "*.bag"))[0]
            reader_utn = bagpy.bagreader(utn_bag_file)
            reader_maryland = bagpy.bagreader(maryland_bag_file)
            res = {"UTN": self.get_times_and_explores(reader_utn), "MARYLAND": self.get_times_and_explores(reader_maryland)}
            return res
        except Exception as ex:
            print(f"An error ocurred on {folder}: {ex}")
            return None
    
    @staticmethod
    def get_explored_df(reader: bagpy.bagreader) -> pd.DataFrame:
        explored_data =  reader.reader.read_messages("/create1/explored")
        amt_of_non_unknown = []
        timestamps = []
        for d in explored_data:
            amt_of_non_unknown.append(d.message.data)
            timestamps.append(d.timestamp.to_sec())
        amt_of_non_unknown = [0.0025 * item for item in amt_of_non_unknown]
        dic = {"timestamp": timestamps, "m_2_explored": amt_of_non_unknown}
        df = pd.DataFrame(dic)
        df["timestamp"] -= dic["timestamp"][0]
        df = df.set_index("timestamp")
        return df

    @staticmethod
    def get_travelled_distance_df(reader: bagpy.bagreader) -> pd.DataFrame:
        travelled_data =  reader.reader.read_messages("/create1/travelled_distance")
        cum_distance = []
        timestamps = []
        for d in travelled_data:
            cum_distance.append(d.message.data)
            timestamps.append(d.timestamp.to_sec())
        dic = {"timestamp": timestamps, "cum_distance": cum_distance}
        df = pd.DataFrame(dic)
        df["timestamp"] -= dic["timestamp"][0]
        df = df.set_index("timestamp")
        return df

    @staticmethod
    def get_trajectory_df(reader: bagpy.bagreader) -> gpd.GeoDataFrame:
        gts =  reader.reader.read_messages("/create1/gts")
        xs = []
        ys = []
        timestamps = []
        for d in gts:
            xs.append(d.message.pose.pose.position.x)
            ys.append(d.message.pose.pose.position.y)
            timestamps.append(d.timestamp.to_sec())
        dic = {"timestamp": timestamps, "x": xs, "y": ys}
        df = pd.DataFrame(dic)
        df["timestamp"] -= dic["timestamp"][0]
        df = df.set_index("timestamp")
        return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y))

    def get_times_and_explores(self, reader: bagpy.bagreader) -> Tuple[List[float], List[float]]:
        explored_df = self.get_explored_df(reader)
        travelled_distance_df = self.get_travelled_distance_df(reader)
        trajectory_df = self.get_trajectory_df(reader)
        return {"explored_df": explored_df, "travelled_distance_df":travelled_distance_df, "trajectory_df":trajectory_df}

    def plot_exp_by_dist(self, index: str):
        res = self.res_values[index]
        utn_df=  pd.merge_asof(res["UTN"]["explored_df"], res["UTN"]["travelled_distance_df"], on="timestamp")
        maryland_df=  pd.merge_asof(res["MARYLAND"]["explored_df"], res["MARYLAND"]["travelled_distance_df"], on="timestamp")
        utn_df = utn_df.dropna()
        maryland_df = maryland_df.dropna()

        plt.plot( utn_df["timestamp"], utn_df["m_2_explored"], label="UTN")
        plt.plot( maryland_df["timestamp"], maryland_df["m_2_explored"], label="MARYLAND")
        plt.ylabel("Explored area [m2]")
        plt.xlabel("Distance travelled [m]")

        plt.legend()
    
    def plot_trajectory(self, index: str):
        report = self.report[index]
        report["UTN"]["trajectory"].buffer(0.2).plot()
        report["MARYLAND"]["trajectory"].buffer(0.2).plot()
        

In [161]:
BASE_DIR = "/home/agusmazzeo/Documents/UTN/Proyecto/outputs"

In [162]:
bdh = BagpyDataHandler(worlds_df, BASE_DIR) 

  0%|                                                                                                         | 0/77 [00:00<?, ?it/s]

[INFO]  Data folder /home/agusmazzeo/Documents/UTN/Proyecto/outputs/evaluation_1/UTN/exploration_data_2022-10-26-12-47-03 already exists. Not creating.
[INFO]  Data folder /home/agusmazzeo/Documents/UTN/Proyecto/outputs/evaluation_1/MARYLAND/exploration_data_2022-10-26-12-47-02 already exists. Not creating.


/tmp/ipykernel_11250/411877001.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
/tmp/ipykernel_11250/411877001.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
  3%|██▌                                                                                              | 2/77 [00:01<01:11,  1.05it/s]

[INFO]  Data folder /home/agusmazzeo/Documents/UTN/Proyecto/outputs/evaluation_5/UTN/exploration_data_2022-10-26-12-47-03 already exists. Not creating.
[INFO]  Data folder /home/agusmazzeo/Documents/UTN/Proyecto/outputs/evaluation_5/MARYLAND/exploration_data_2022-10-26-12-47-03 already exists. Not creating.


/tmp/ipykernel_11250/411877001.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
/tmp/ipykernel_11250/411877001.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
  4%|███▊                                                                                             | 3/77 [00:05<02:33,  2.07s/it]

...

/tmp/ipykernel_11250/411877001.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
/tmp/ipykernel_11250/411877001.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
  9%|████████▊                                                                                        | 7/77 [00:15<02:56,  2.52s/it]

An error ocurred on /home/agusmazzeo/Documents/UTN/Proyecto/outputs/evaluation_2: list index out of range
[INFO]  Data folder /home/agusmazzeo/Documents/UTN/Proyecto/outputs/evaluation_8/UTN/exploration_data_2022-10-26-14-27-24 already exists. Not creating.
[INFO]  Data folder /home/agusmazzeo/Documents/UTN/Proyecto/outputs/evaluation_8/MARYLAND/exploration_data_2022-10-26-14-26-27 already exists. Not creating.


/tmp/ipykernel_11250/411877001.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
/tmp/ipykernel_11250/411877001.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
 12%|███████████▎                                                                                     | 9/77 [00:16<01:51,  1.64s/it]

[INFO]  Data folder /home/agusmazzeo/Documents/UTN/Proyecto/outputs/evaluation_10/UTN/exploration_data_2022-10-26-14-46-41.orig already exists. Not creating.
[INFO]  Data folder /home/agusmazzeo/Documents/UTN/Proyecto/outputs/evaluation_10/MARYLAND/exploration_data_2022-10-26-14-38-17 already exists. Not creating.


/tmp/ipykernel_11250/411877001.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
/tmp/ipykernel_11250/411877001.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  report[key] = {"trajectory": poly.append(result[key]["trajectory_df"]), **self.get_exploration_data(key, index, result)}
 13%|████████████▍                                                                                   | 10/77 [00:20<02:17,  2.05s/it]


In [173]:
pd.DataFrame(bdh.report['evaluation_3']).T

,trajectory,explored_percentage,total_time_min,dist_explored_20%,dist_explored_50%,dist_explored_80%,dist_total
UTN,...,99.476348,421,6.562366,42.59753,72.570625,237.277527
MARYLAND,...,99.387183,207,9.798131,30.779104,55.964676,127.377296


In [ ]:
reform = {(outerKey, innerKey): values for outerKey, innerDict in dictionary.iteritems() for innerKey, values in innerDict.iteritems()}

In [153]:
multi_index = pd.MultiIndex.from_product([bdh.report.keys(), ["UTN", "MARYLAND"]], names=["world", "algorithm"])
multi_index

MultiIndex([( 'evaluation_1',      'UTN'),
            ( 'evaluation_1', 'MARYLAND'),
            ( 'evaluation_5',      'UTN'),
            ( 'evaluation_5', 'MARYLAND'),
            ( 'evaluation_0',      'UTN'),
            ( 'evaluation_0', 'MARYLAND'),
            ( 'evaluation_9',      'UTN'),
            ( 'evaluation_9', 'MARYLAND'),
            ( 'evaluation_6',      'UTN'),
            ( 'evaluation_6', 'MARYLAND'),
            ( 'evaluation_3',      'UTN'),
            ( 'evaluation_3', 'MARYLAND'),
            ( 'evaluation_8',      'UTN'),
            ( 'evaluation_8', 'MARYLAND'),
            ('evaluation_10',      'UTN'),
            ('evaluation_10', 'MARYLAND')],
           names=['world', 'algorithm'])

In [159]:
pd.DataFrame(bdh.report, index=multi_index, orient='index')

TypeError: from_dict() got an unexpected keyword argument 'index'